In [32]:
import pandas as pd
import numpy as np
import ta
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [33]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(500) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
df.iloc[:,1:] = df.iloc[:, 1:].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [36]:
df['SMA'] = df['Close'].rolling(20).mean()
df['RSI'] = ta.momentum.rsi(df['Close'])
df['MACD'] = ta.trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)

,Open,High,Low,Close,Volume,SMA,RSI,MACD,Momentum
Time,,,,,,,,,
2024-07-10 01:36:00,57959.99,57963.77,57901.99,57902.0,19.71896,NaN,NaN,NaN,NaN
2024-07-10 01:37:00,57901.99,57928.21,57880.0,57880.01,16.0797,NaN,NaN,NaN,NaN
2024-07-10 01:38:00,57880.01,57898.0,57856.13,57856.14,9.38101,NaN,NaN,NaN,NaN
2024-07-10 01:39:00,57856.13,57865.39,57814.0,57830.01,7.65298,NaN,NaN,NaN,NaN
2024-07-10 01:40:00,57830.01,57830.01,57808.0,57822.73,6.25104,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-07-10 09:51:00,58572.01,58620.0,58570.0,58596.01,11.27354,58482.7570,60.761707,22.622438,-0.001571
2024-07-10 09:52:00,58596.01,58596.01,58541.57,58568.0,23.12234,58489.2805,55.729149,20.227220,-0.001235
2024-07-10 09:53:00,58567.99,58574.0,58556.0,58570.0,5.66524,58496.0805,56.009319,17.921853,0.0
